In [1]:
import torch
from process.data import MPIIDataset
from model.modules import FCBlock,MixerLayer
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

In [2]:
# -----------------------------
# CONFIG
# -----------------------------
IMAGE_DIR = '../images/'
CSV_PATH = '../mpii_human_pose_v1_u12_2/mpii_human_pose.csv'
IMG_SIZE = 224
NUM_JOINTS = 16

In [3]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [4]:
mpii = MPIIDataset(CSV_PATH,IMAGE_DIR,transform)

In [5]:
mpii[0][1].shape

torch.Size([16, 2])

In [6]:
data = DataLoader(mpii,batch_size=16,shuffle=True)

In [7]:
from model.encoder import CompositionalEncoder,VectorQuantizer,Decoder
import torch

random_tensor = torch.randn(size=(1,16,2))
with torch.no_grad():
    encoder = CompositionalEncoder(k=16,d=2,h=256,m=16)
    vq = VectorQuantizer(v=512,h=256,commitment_cost=0.25)
    decoder = Decoder(k=16,d=2,h=256,m=16)
    output = encoder(random_tensor)
    print(output.shape)
    output = vq(output)[0]
    print(output.shape)
    output = decoder(output)
    print(output.shape)

torch.Size([1, 16, 256])
torch.Size([1, 16, 256])
torch.Size([1, 16, 2])


In [4]:
# Increasing the number of entries V in the
# codebook decreases the quantization error. However, it also
# increases the classification difficulty as the number of categories becomes larger.
from model.backbone import SwinTransformerV2
import torch
model = SwinTransformerV2(
    img_size=256,
    patch_size=4,
    in_chans=3,
    num_classes=0,  # Set to 0 to extract features instead of logits
    embed_dim=96,
    depths=[2, 2, 6, 2],
    num_heads=[3, 6, 12, 24],
    window_size=7,
    ape=False,
    patch_norm=True,
    use_checkpoint=False
)

# Load a 256x256 RGB image tensor
dummy_input = torch.randn(1, 3, 256, 256)  # Batch size 1, 3 channels

# Extract features from the model (this bypasses the classifier head)
features = model.forward_features(dummy_input)
print(features.shape)  # Expecting [1, feature_dim]

RuntimeError: shape '[1, 9, 7, 9, 7, 1]' is invalid for input of size 4096